In [1]:
# ==============================================================================
# STEP 1: INSTALL LIBRARIES IN KAGGLE
# ==============================================================================
!pip install "ray[tune]"
!pip install cudf-cu12 cuml-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install lightgbm

# ==============================================================================
# STEP 2: IMPORT LIBRARIES
# ==============================================================================
import os
import warnings
import pandas as pd
import cudf
from cuml.model_selection import train_test_split
from sklearn.metrics import r2_score
import ray
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler
from lightgbm import LGBMRegressor

warnings.filterwarnings("ignore")

# ==============================================================================
# STEP 3: DEFINE THE RAY TUNE TRAINABLE FUNCTION (LightGBM GPU)
# ==============================================================================
def train_lgbm(config, data):
    """
    Trainable function for Ray Tune using LightGBM GPU.
    """
    X_train, X_test, y_train, y_test = data

    # Convert cudf to pandas for LightGBM if needed
    X_train_pd = X_train.to_pandas()
    X_test_pd = X_test.to_pandas()
    y_train_pd = y_train.to_pandas()
    y_test_pd = y_test.to_pandas()

    model = LGBMRegressor(
        n_estimators=config["n_estimators"],
        num_leaves=config["num_leaves"],
        learning_rate=config["learning_rate"],
        max_depth=config["max_depth"],
        min_data_in_leaf=config["min_data_in_leaf"],
        feature_fraction=config["feature_fraction"],
        bagging_fraction=config["bagging_fraction"],
        bagging_freq=config["bagging_freq"],
        lambda_l1=config["lambda_l1"],
        lambda_l2=config["lambda_l2"],
        device_type="gpu",
        random_state=42
    )
    model.fit(X_train_pd, y_train_pd)

    y_pred = model.predict(X_test_pd)
    score = r2_score(y_test_pd, y_pred)
    train.report({"r2_score": float(score)})

# ==============================================================================
# STEP 4: MAIN HYPERPARAMETER TUNING SCRIPT
# ==============================================================================
def run_gpu_tuning(
    data_file: str,
    num_samples: int = 100
):
    """
    Loads data, defines search space, and runs Ray Tune on GPU LightGBM.
    """
    print("\n" + "="*80)
    print(" SRP MODEL GPU HYPERPARAMETER TUNING (LIGHTGBM) ".center(80, "="))
    print("="*80)

    # --- 1. Load Data ---
    print(f"\nLoading data from: {data_file}")
    if not os.path.exists(data_file):
        print(f"ERROR: Data file not found at '{data_file}'")
        return

    pdf = pd.read_excel(data_file)
    gdf = cudf.from_pandas(pdf)
    print(f"✓ Loaded {len(gdf)} rows and converted to GPU DataFrame.")

    target_cols = ["srp_ax_mps2", "srp_ay_mps2", "srp_az_mps2"]
    feature_cols = [
        'A_OVER_M', 'CR_eff', 'AU2_over_R2', 'shadow_factor',
        'sun_to_sat_ux', 'sun_to_sat_uy', 'sun_to_sat_uz'
    ]

    X = gdf[feature_cols]

    # Shared LightGBM search space
    search_space = {
        "n_estimators": tune.randint(200, 1000),
        "num_leaves": tune.randint(16, 128),
        "learning_rate": tune.loguniform(1e-3, 1e-1),
        "max_depth": tune.randint(-1, 20),
        "min_data_in_leaf": tune.randint(10, 200),
        "feature_fraction": tune.uniform(0.6, 1.0),
        "bagging_fraction": tune.uniform(0.5, 1.0),
        "bagging_freq": tune.randint(0, 5),
        "lambda_l1": tune.loguniform(1e-9, 1e-1),
        "lambda_l2": tune.loguniform(1e-9, 1e-1),
    }

    scheduler = ASHAScheduler(metric="r2_score", mode="max")
    all_best_params = {}

    # --- 2. Loop per target ---
    for target_col in target_cols:
        print("\n" + "─"*80)
        print(f" Tuning for: {target_col} ".center(80, "─"))
        print("─"*80)

        y = gdf[target_col]
        X_train, X_test, y_train_target, y_test_target = train_test_split(
            X, y, test_size=0.3, random_state=42
        )

        data_id = ray.put((X_train, X_test, y_train_target, y_test_target))
        trainable_with_data = tune.with_parameters(train_lgbm, data=data_id)

        analysis = tune.run(
            trainable_with_data,
            resources_per_trial={"cpu": 1, "gpu": 1},
            config=search_space,
            num_samples=num_samples,
            scheduler=scheduler,
            verbose=1,
            name=f"srp_gpu_tune_{target_col}"
        )

        best_trial = analysis.get_best_trial("r2_score", "max", "last")
        best_params = best_trial.config
        all_best_params[target_col] = best_params

        print(f"\n✓ Tuning complete for {target_col}")
        print(f"  -> Best R² score: {best_trial.last_result['r2_score']:.6f}")
        print(f"  -> Best hyperparameters: {best_params}")

    # --- 3. Summary ---
    print("\n" + "="*80)
    print(" TUNING SUMMARY ".center(80, "="))
    print("="*80)
    print("\nBest hyperparameters found for each target:\n")
    for target, params in all_best_params.items():
        print(f"Target: {target}")
        print(f"  Params: {params}\n")

    print("Use these parameters to train final LightGBM models locally.")

# ==============================================================================
# STEP 5: EXECUTE
# ==============================================================================
if __name__ == "__main__":
    if ray.is_initialized():
        ray.shutdown()
    ray.init()

    try:
        kaggle_data_path = "/kaggle/input/train-lgb/training_data_combined.xlsx"
        run_gpu_tuning(
            data_file=kaggle_data_path,
            num_samples=100
        )
    finally:
        ray.shutdown()

(train_lgbm pid=11169) [LightGBM] [Info] This is the GPU trainer!!
(train_lgbm pid=11169) [LightGBM] [Info] Total Bins 1294
(train_lgbm pid=11169) [LightGBM] [Info] Number of data points in the train set: 15686, number of used features: 7
(train_lgbm pid=11107) [LightGBM] [Info] Size of histogram bin entry: 8 [repeated 109x across cluster]
(train_lgbm pid=11107) [LightGBM] [Info] 7 dense feature groups (0.12 MB) transferred to GPU in 0.000625 secs. 0 sparse feature groups [repeated 109x across cluster]
(train_lgbm pid=11169) [LightGBM] [Warning] min_data_in_leaf is set=135, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=135 [repeated 3x across cluster]
(train_lgbm pid=11169) [LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0 [repeated 3x across cluster]
(train_lgbm pid=11169) [LightGBM] [Warning] feature_fraction is set=0.8610859104582816, colsample_bytree=1.0 will be ignored. Current value: feature_fractio

(train_lgbm pid=11169) /usr/local/lib/python3.11/dist-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_lgbm pid=11169)   _log_deprecation_warning(


(train_lgbm pid=11170) [LightGBM] [Info] This is the GPU trainer!!
(train_lgbm pid=11170) [LightGBM] [Info] Total Bins 1294
(train_lgbm pid=11170) [LightGBM] [Info] Number of data points in the train set: 15686, number of used features: 7
(train_lgbm pid=11170) [LightGBM] [Info] Size of histogram bin entry: 8 [repeated 788x across cluster]
(train_lgbm pid=11170) [LightGBM] [Info] 7 dense feature groups (0.12 MB) transferred to GPU in 0.000818 secs. 0 sparse feature groups [repeated 788x across cluster]
(train_lgbm pid=11169) [LightGBM] [Warning] min_data_in_leaf is set=135, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=135 [repeated 3x across cluster]
(train_lgbm pid=11169) [LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0 [repeated 3x across cluster]
(train_lgbm pid=11169) [LightGBM] [Warning] feature_fraction is set=0.8610859104582816, colsample_bytree=1.0 will be ignored. Current value: feature_fractio

(train_lgbm pid=11170) /usr/local/lib/python3.11/dist-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_lgbm pid=11170)   _log_deprecation_warning(


(train_lgbm pid=11273) [LightGBM] [Info] This is the GPU trainer!!
(train_lgbm pid=11273) [LightGBM] [Info] Total Bins 1294
(train_lgbm pid=11273) [LightGBM] [Info] Number of data points in the train set: 15686, number of used features: 7
(train_lgbm pid=11170) [LightGBM] [Info] Size of histogram bin entry: 8 [repeated 66x across cluster]
(train_lgbm pid=11170) [LightGBM] [Info] 7 dense feature groups (0.12 MB) transferred to GPU in 0.000821 secs. 0 sparse feature groups [repeated 66x across cluster]
(train_lgbm pid=11273) [LightGBM] [Warning] min_data_in_leaf is set=129, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=129 [repeated 3x across cluster]
(train_lgbm pid=11273) [LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1 [repeated 3x across cluster]
(train_lgbm pid=11273) [LightGBM] [Warning] feature_fraction is set=0.9976659910218765, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=

...

(train_lgbm pid=16406) [LightGBM] [Info] This is the GPU trainer!! [repeated 2x across cluster]
(train_lgbm pid=16406) [LightGBM] [Info] Total Bins 1294 [repeated 2x across cluster]
(train_lgbm pid=16406) [LightGBM] [Info] Number of data points in the train set: 15686, number of used features: 7 [repeated 2x across cluster]
(train_lgbm pid=16406) [LightGBM] [Warning] min_data_in_leaf is set=57, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=57 [repeated 6x across cluster]
(train_lgbm pid=16406) [LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2 [repeated 6x across cluster]
(train_lgbm pid=16406) [LightGBM] [Warning] feature_fraction is set=0.7425278313172838, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7425278313172838 [repeated 6x across cluster]
(train_lgbm pid=16406) [LightGBM] [Warning] bagging_fraction is set=0.984783092379764, subsample=1.0 will be ignored. Current value: b

(train_lgbm pid=16406) /usr/local/lib/python3.11/dist-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454 [repeated 2x across cluster]
(train_lgbm pid=16406)   _log_deprecation_warning( [repeated 2x across cluster]


(train_lgbm pid=16420) [LightGBM] [Info] This is the GPU trainer!!
(train_lgbm pid=16420) [LightGBM] [Info] Total Bins 1294
(train_lgbm pid=16420) [LightGBM] [Info] Number of data points in the train set: 15686, number of used features: 7
(train_lgbm pid=16406) [LightGBM] [Warning] min_data_in_leaf is set=57, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=57 [repeated 3x across cluster]
(train_lgbm pid=16406) [LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2 [repeated 3x across cluster]
(train_lgbm pid=16406) [LightGBM] [Warning] feature_fraction is set=0.7425278313172838, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7425278313172838 [repeated 3x across cluster]
(train_lgbm pid=16406) [LightGBM] [Warning] bagging_fraction is set=0.984783092379764, subsample=1.0 will be ignored. Current value: bagging_fraction=0.984783092379764 [repeated 3x across cluster]
(train_lgbm pid=16406) [

2025-11-03 19:50:30,786	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/srp_gpu_tune_srp_az_mps2' in 0.0375s.
2025-11-03 19:50:30,818	INFO tune.py:1041 -- Total run time: 587.75 seconds (587.67 seconds for the tuning loop).
(train_lgbm pid=16420) /usr/local/lib/python3.11/dist-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_lgbm pid=16420)   _log_deprecation_warning(



✓ Tuning complete for srp_az_mps2
  -> Best R² score: 0.999665
  -> Best hyperparameters: {'n_estimators': 391, 'num_leaves': 55, 'learning_rate': 0.03482361164415041, 'max_depth': 11, 'min_data_in_leaf': 53, 'feature_fraction': 0.9593848587695054, 'bagging_fraction': 0.6711681972475381, 'bagging_freq': 0, 'lambda_l1': 4.26394757201816e-09, 'lambda_l2': 4.140014558426177e-05}

================================ TUNING SUMMARY ================================

Best hyperparameters found for each target:

Target: srp_ax_mps2
  Params: {'n_estimators': 778, 'num_leaves': 37, 'learning_rate': 0.018303631950642548, 'max_depth': 11, 'min_data_in_leaf': 69, 'feature_fraction': 0.9986531777035408, 'bagging_fraction': 0.8574119097776769, 'bagging_freq': 4, 'lambda_l1': 1.5058169020871542e-09, 'lambda_l2': 0.01630692236861022}

Target: srp_ay_mps2
  Params: {'n_estimators': 977, 'num_leaves': 50, 'learning_rate': 0.029626981002240736, 'max_depth': 7, 'min_data_in_leaf': 55, 'feature_fraction': 0.